In [ ]:
import requests
import numpy as np
import pandas as pd
from bs4 import BeautifulSoup


# 懂车帝搜索URL：https://www.dongchedi.com/search?keyword=%E9%95%BF%E5%AE%89CS55PLUS&currTab=1&city_name=%E6%9D%AD%E5%B7%9E&search_mode=common
front_url = "https://www.dongchedi.com/search?keyword="
back_url = "&currTab=1&city_name=%E6%9D%AD%E5%B7%9E&search_mode=common"

def askurl(url):
    headers = {
        "user-agent": "Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/86.0.4240.198 Safari/537.36 QIHU 360SE"
    }
    html = requests.get(url, headers=headers)
    return html.text

# 分割字符串：10.12-18.53万：[10.12, 18.53]
def split_str(str):
    site_1 = str.find('-')
    str1 = str[:site_1]
    site_2 = len(str)
    str2 = str[site_1+1:site_2-1]
    return str1, str2

def getcars():
    all_brands = []
    competitive_brand = pd.read_csv('竞品.csv')
    brands = competitive_brand['吉利车型'].drop_duplicates()
    for name in brands:
        jili_brand = competitive_brand[:][competitive_brand['吉利车型'] == name].iloc[0]
        # 获取懂车帝中吉利汽车的竞品
        html = askurl(front_url + name + back_url)
        soup = BeautifulSoup(html, 'html.parser')
        section = soup.find('section', class_='jsx-2188849952 level-car vertical')
        if section is None: continue
        div = section.find_all('div', class_='jsx-2188849952 info')
        for i in range(len(div)):
            other_brand = div[i].find('a').text
            other_price = div[i].find('div', class_='jsx-2188849952 price').text
            low_price, high_price = split_str(other_price)
            if low_price == '暂': continue
            all_brands.append([name, jili_brand[2], jili_brand[3], other_brand, low_price, high_price])
    all_brands = np.array(all_brands)
    all_brands = pd.DataFrame({'吉利车型': all_brands[:, 0], '低配价格': all_brands[:, 1], '高配价格': all_brands[:, 2], '其它车型': all_brands[:, 3], '其它车型低配价格': all_brands[:, 4], '其它车型高配价格': all_brands[:, 5]})
    all_brands.to_csv("懂车帝.csv")


getcars()